# Securing an atoti session with Auth0 using Google

Securing a session comes in two parts:
1. Implementing authentication mechanism to secure access to the session
2. Restricting access of modules or data access by users based on the roles granted

With the [Atoti+ plugin](https://docs.atoti.io/latest/atoti_plus.html), atoti supports multiple [authentication mechanisms](https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.html) to cater to the needs of our end users. This notebook demonstrates how we use [OpenID Connect (OIDC)](https://openid.net/connect/) through [Google Cloud credentials](https://cloud.google.com/docs/authentication) to authenticate access to atoti web application.

We will explore security with Atoti+ using the [Top 50 Fast Food](https://www.kaggle.com/datasets/stetsondone/top50fastfood) dataset from Kaggle, combined with its parent company information sourced from the internet.  


__Note__:  

The notebook is structured in this order:
1. Authentication setup during session instantiation
2. Create BI analytics platform with atoti Community Edition
3. Users and roles management with Atoti+

Mainly, we look at the roles last because we need to know the names of the tables and columns which we want to impose restrictions on.  
Thereafter, we can create the roles with restrictions. Check out [atoti documentation](https://docs.atoti.io) to read more on [securing the session](https://docs.atoti.io/latest/how_tos/security/secure_a_session.html#Configuring-the-authentication-mechanism).  

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=oidc-google" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover.png" alt="Try atoti"></a></div>

In [1]:
import os

import atoti as tt
from atoti_plus import UserServiceClient

## 1. Authentication setup in atoti  

Google Cloud APIs use the [OAuth 2.0 protocol](https://tools.ietf.org/html/rfc6749) for authenticating the user accounts. We demonstrate below how we configure the authentication parameter of [`atoti.Session`](https://docs.atoti.io/latest/lib/atoti/atoti.session.html#atoti.Session.__init__) to connect Google Cloud Credential API using OIDC.

Firstly, we have to configure the _OAuth consent screen_ and registered our app. Remember to set the "User type" to _Internal_, so that our app is limited to Google Workspace users within our organisation.  

<img src="https://data.atoti.io/notebooks/security/img/google-oauth-consent-screen.png" width="70%" />

Proceed to create a credential of type "OAuth client IDs" under __APIs & Services > Credentials__ in Google Cloud.  
Thereafter, refer to the following for the setting mapping against google:  
<img src="https://data.atoti.io/notebooks/security/img/google-setup.png" width="70%" />

In [2]:
authentication = tt.OidcConfig(
    provider_id="google",
    issuer_url="https://accounts.google.com",
    client_id=os.environ["GOOGLE_CLIENT_ID"],
    client_secret=os.environ["GOOGLE_CLIENT_SECRET"],
    scopes=["https://www.googleapis.com/auth/userinfo.email"],
    name_claim="email",
)

Client ids and secrets should be kept private. As suggested in the [documentation](https://docs.atoti.io/latest/how_tos/security/secure_a_session.html#Configuring-the-authentication-mechanism), connection details can be read from environment variables for improved security.  

### 1.1. Instantiating session with fixed port

It is important to fix the port for atoti web application as we will require it to configure the authorized redirect URIs for the OAuth 2.0 Client ID in Google Cloud.

<img src="https://data.atoti.io/notebooks/security/img/google-application-url.png" width="70%" />  

In Google Cloud, remember to include the following redirect URI with the [provider id](https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.oidc_config.html#atoti.OidcConfig.provider_id) set to `google`:

`f"{session_url}/login/oauth2/code/{provider_id}"`

In [3]:
session = tt.Session(
    port=10011, authentication=authentication, user_content_storage="./content"
)

From [atoti v0.7.2](https://docs.atoti.io/latest/releases/0.7.2.html#added) onwards, atoti uses [UserServiceClient](https://docs.atoti.io/latest/lib/atoti-plus/atoti_plus.user_service_client.user_service_client.html#atoti_plus.UserServiceClient) to manage the dynamic aspects of the security configuration.  
With the client, multiple sessions can be configured with the same security configuration stored on the _user content storage_. The security configuration covers the roles and restrictions.

In [4]:
user_service_client = UserServiceClient.from_session(session)

## 2. Create BI analytics platform with atoti Community Edition

Once the session is created, we can proceed with the usual data loading into atoti table, cube and measures creation.  
Remember to re-execute these cells if you have changed the mode of authentiction.

### 2.1 Table creation

Although we can [`create table`](https://docs.atoti.io/latest/lib/atoti/atoti.session.html#atoti.Session.create_table) before loading data in, we used `read_csv` in our example to create and load data into the atoti tables.

In [5]:
base_tbl = session.read_csv(
    "s3://data.atoti.io/notebooks/security/data/parent_co.csv",
    table_name="parent_co",
    keys=["company", "parent_company"],
    process_quotes=True,
)
base_tbl.head()

,,name
parent_company,company,
Inspire Brands,arbys,Arby's
"Domino's Pizza, Inc.",dominos,Domino's
"Papa Murphy's Holdings, Inc.",papa_murphys,Papa Murphy's
Focus Brands,auntie_annes,Auntie Anne's
Panda Restaurant Group,panda_express,Panda Express


In [6]:
enrichment_tbl = session.read_csv(
    "s3://data.atoti.io/notebooks/security/data/top_50_fast_food_US.csv",
    table_name="top_50",
    keys=["company"],
)
enrichment_tbl.head()

,category,sales_in_millions_2019,sales_per_unit_thousands_2019,franchised_units_2019,company_owned_units_2019,total_units_2019,unit_change_from_2018
company,,,,,,,
popeyes_chicken,chicken,3750,1541,2458,41,2499,131
del_taco,global,850,1554,296,300,596,16
jack_in_the_box,burger,3505,1565,2106,137,2243,6
tim_hortons,sandwich,840,1165,715,0,715,-12
mcdonalds,burger,40413,2912,13154,692,13846,-66


In [7]:
base_tbl.join(enrichment_tbl)

### 2.2. Cube creation

In [8]:
cube = session.create_cube(base_tbl, "Fast food analysis")

### 2.3 Measures creation

In [9]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

In [10]:
m["sales_in_millions"] = tt.agg.sum(enrichment_tbl["sales_in_millions_2019"])
m["sales_per_unit_thousands"] = tt.agg.sum(
    enrichment_tbl["sales_per_unit_thousands_2019"]
)
m["franchised_units"] = tt.agg.sum(enrichment_tbl["franchised_units_2019"])
m["company_owned_units"] = tt.agg.sum(enrichment_tbl["company_owned_units_2019"])
m["total_units"] = tt.agg.sum(enrichment_tbl["total_units_2019"])
m["unit_change_from_2018"] = tt.agg.sum(enrichment_tbl["unit_change_from_2018"])

In [11]:
m["% franchised"] = m["franchised_units"] / m["total_units"]
m["% franchised"].formatter = "DOUBLE[0.00%]"

## 3. Users and roles management with Atoti+

### 3.1. atoti reserved roles  

The below roles are reserved in atoti and should not be altered by users:
- ROLE_ADMIN: able to access all objects in the web application
- ROLE_USER: able to access all data by default. Access to objects such as dashboards, folders, widgets etc is only upon sharing access granted to role.
- ROLE_SHARE: able to share objects such as dashboards, folders, widgets and filters.  

__All users, including the administrator, require the role *ROLE\_USER* to be able to access the atoti web application.__

Let's assume both users, *atoti\_admin* and *atoti\_user*, have been granted the role __ROLE_USER__ and *atoti\_admin* is also granted the role __ROLE_ADMIN__.  
While both *atoti\_admin* and *atoti\_user* are able to access all data, *atoti\_admin* is able to access all objects such as folders and dashboards.  
*atoti\_user* is able to access only the objects created by the user him/herself. Objects created by other users can only be access upon shared access granted.  

Both *atoti\_admin* and *atoti\_user* will not be able to share objects (via the "Share" icon as shown below) unless granted the role __ROLE_SHARE__.  

<img src="https://data.atoti.io/notebooks/security/img/sharing.png" width="70%"/>

### 3.2. Role creation with restrictions  

Data restriction is based on users' requirement. In our use case, we assumed two groups of users with data access limited to those of their parent company:
- users belonging to parent company _Inspire Brands_
- users belonging to parent company _Restaurant Brands International Inc._

Therefore, we will create two roles to apply the restrictions based on the `parent_company` column from the `parent_co` table.  
We will define key that is a tuple, consisting of the name of the table and its column, along with the restricted values imposed on it. 

__NOTE:__  
- We can skip role creation if there are no restrictions imposed on the role. 
- The value provided under the restrictions is cap-sensitive.

In [12]:
role_inspire = user_service_client.create_role(
    "ATOTI_ROLE_INSPIRE",
    restrictions={("parent_co", "parent_company"): ["Inspire Brands"]},
)

role_restaurant = user_service_client.create_role(
    "ATOTI_ROLE_RESTAURANT",
    restrictions={
        ("parent_co", "parent_company"): ["Restaurant Brands International Inc."]
    },
)

#### 3.2.1. Restricted access from combination of roles

Multiple roles can be assigned to the same user. To demonstrate how the access will change when this happens, we create some other roles that restrict data access by the restaurant category, i.e. column `category` from the table `top_50`.

In [13]:
role_team_burger = user_service_client.create_role(
    "ATOTI_ROLE_BURGER", restrictions={("top_50", "category"): ["burger"]}
)

role_team_sandwich = user_service_client.create_role(
    "ATOTI_ROLE_SANDWICH", restrictions={("top_50", "category"): ["sandwich"]}
)

role_team_snack = user_service_client.create_role(
    "ATOTI_ROLE_SNACK", restrictions={("top_50", "category"): ["snack"]}
)

When combined with the restricted role on the `parent_company`, user's access will be further restricted to based on the restriction of the added role.  

For instance, users who are assigned the role __ATOTI_ROLE_BURGER__ will be able to access all the data under _burger_ category restaurants, regardless of the parent companies.  

However, when the same user is also granted the role __ATOTI_ROLE_INSPIRE__, then the user can only access data of restaurants under parent company _Inspire Brands_ that is of category _burger_. 

### 3.3. Role assignment in atoti  

__atoti use Google for authentication only.__ Roles are created and assigned to the users in atoti and they can be granted without having to restart the application.

#### 3.3.1. Assign default roles to authenticated users

We could default the roles such as __ROLE_SHARE__ for all users who logged in successfully. 

In [14]:
user_service_client.oidc.default_roles.update(["ROLE_SHARE"])

__BE CAUTION__ when granting __ROLE_USER__ to users by default as users with this role will be able to access the application and the available data.  

We want to prevent unwanted access by people outside of the organisation. Therefore, this should only be done when the user type for the app registered under the _OAuth consent screen_ is ___Internal___. Otherwise, it is better to grant __ROLE_USER__ individually to the rightful users in the next step.

#### 3.3.2. Granting roles to users authenticated through Google credential

Instead of mapping the roles from Authentication Provider as in the case of Auth0, we use the user's email address to map to the assigned atoti roles.

In [15]:
ROLE_USER = "ROLE_USER"

user_service_client.oidc.role_mapping.update(
    {
        "inspire_m@test.com": [ROLE_USER, role_inspire.name],
        "inspire_user1@test.com": [ROLE_USER, role_inspire.name, role_team_burger.name],
        "inspire_user2@test.com": [
            ROLE_USER,
            role_inspire.name,
            role_team_sandwich.name,
            role_team_snack.name,
        ],
        "restaurant_m@test.com": [ROLE_USER, role_restaurant.name],
        "restaurant_user1@test.com": [
            ROLE_USER,
            role_restaurant.name,
            role_team_burger.name,
        ],
        "restaurant_user2@test.com": [
            ROLE_USER,
            role_restaurant.name,
            role_team_sandwich.name,
            role_team_snack.name,
        ],
    }
)

## 4. Test login and access management in web application

Try out any of these users:

___Administrator___
- atoti_admin

___Generic user___
- atoti_user

___Inspire Brands users___
- Inspire_user1 (Access for restaurants of category Burger only)
- Inspire_user2 (Access for restaurants of category Sandwich and Snack only)
- Inspire_manager

___Restaurant Brands International LLC users___
- Restaurant_user1 (Access for restaurants of category Burger only)
- Restaurant_user2 (Access for restaurants of category Sandwich and Snack only)
- Restaurant_manager

In [16]:
session.link()

Open the notebook in JupyterLab with the atoti extension enabled to see this link.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=oidc-google" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover-try.png" alt="Try atoti"></a></div>